## Prepair

In [1]:
## config.yamlの読み込み
import yaml
with open("../scripts/config.yaml", "r", encoding='utf-8') as file:
    config = yaml.safe_load(file)

In [2]:
import gc
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import random
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, cohen_kappa_score
from lightgbm import log_evaluation, early_stopping
import polars as pl
from pathlib import Path
import sys
import torch

In [3]:
# 自作関数の読み込み
repo_dir = Path().resolve().parents[0]
root_dir = Path().resolve().parents[1]
s3_dir = root_dir / "s3storage/01_public/auto_essay_scorer_lab2/data/"
sys.path.append(str(repo_dir / "scripts/"))
from utils.path import PathManager
from utils.data import *
from utils.model import *

## パスの設定
mode = config["model_name"]
path_to = PathManager(s3_dir, mode)

2024-05-30 00:08:49.889173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 00:08:49.889275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 00:08:49.891844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 00:08:51.576895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# a_valueとb_valueを確認

In [4]:
## 学習データ読み込み
load_path = path_to.train_all_mart_dir
train_data = pl.read_csv(load_path)

In [5]:
# a_value : 2.998
mean_score = train_data['score'].mean()
print(f"Average of column 'score': {mean_score}")

# b_value : 1.092
variance_score = train_data['score'].var()
print(f"Variance of column 'score': {variance_score}")

Average of column 'score': 2.948402380539666
Variance of column 'score': 1.0918134361390224


# scripts/01_prepare.py 実行後のデータ確認

In [30]:
## データの読み込み
fold_num = "0"

train_path: Path = path_to.add_meta_mart_dir / f'fold_{fold_num}/train_fold_add_meta.csv'
train_data = pl.read_csv(train_path)
valid_path: Path = path_to.add_meta_mart_dir / f'fold_{fold_num}/valid_fold_add_meta.csv'
valid_data = pl.read_csv(valid_path)

In [31]:
train_data.head(1)

,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,paragraph_>300_cnt,paragraph_>350_cnt,paragraph_>400_cnt,paragraph_>500_cnt,paragraph_>600_cnt,paragraph_>700_cnt,paragraph_<25_cnt,paragraph_<49_cnt,paragraph_error_num_max,paragraph_len_max,paragraph_sentence_cnt_max,paragraph_word_cnt_max,paragraph_error_num_mean,paragraph_len_mean,paragraph_sentence_cnt_mean,paragraph_word_cnt_mean,paragraph_error_num_min,paragraph_len_min,paragraph_sentence_cnt_min,paragraph_word_cnt_min,paragraph_error_num_sum,paragraph_len_sum,paragraph_sentence_cnt_sum,paragraph_word_cnt_sum,paragraph_error_num_first,paragraph_len_first,…,tfid_cnt_2136,tfid_cnt_2137,tfid_cnt_2138,tfid_cnt_2139,tfid_cnt_2140,tfid_cnt_2141,tfid_cnt_2142,tfid_cnt_2143,tfid_cnt_2144,tfid_cnt_2145,tfid_cnt_2146,tfid_cnt_2147,tfid_cnt_2148,tfid_cnt_2149,tfid_cnt_2150,tfid_cnt_2151,tfid_cnt_2152,tfid_cnt_2153,tfid_cnt_2154,tfid_cnt_2155,tfid_cnt_2156,tfid_cnt_2157,tfid_cnt_2158,tfid_cnt_2159,tfid_cnt_2160,tfid_cnt_2161,tfid_cnt_2162,tfid_cnt_2163,tfid_cnt_2164,tfid_cnt_2165,tfid_cnt_2166,tfid_cnt_2167,tfid_cnt_2168,tfid_cnt_2169,tfid_cnt_2170,tfid_cnt_2171,tfid_cnt_2172
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,"""0040e27""",5,5,5,5,5,5,5,4,4,2,1,1,0,0,0,0,0,2,424,7,80,1.4,300.0,5.6,55.2,1,179,5,31,7,1500,28,276,1,179,…,0,2,0,0,0,2,0,0,0,0,0,0,1,0,1,1,0,0,2,0,1,1,1,12,0,12,0,7,7,0,0,0,0,0,0,0,0


In [32]:
valid_data.head(1)

,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,paragraph_>300_cnt,paragraph_>350_cnt,paragraph_>400_cnt,paragraph_>500_cnt,paragraph_>600_cnt,paragraph_>700_cnt,paragraph_<25_cnt,paragraph_<49_cnt,paragraph_error_num_max,paragraph_len_max,paragraph_sentence_cnt_max,paragraph_word_cnt_max,paragraph_error_num_mean,paragraph_len_mean,paragraph_sentence_cnt_mean,paragraph_word_cnt_mean,paragraph_error_num_min,paragraph_len_min,paragraph_sentence_cnt_min,paragraph_word_cnt_min,paragraph_error_num_sum,paragraph_len_sum,paragraph_sentence_cnt_sum,paragraph_word_cnt_sum,paragraph_error_num_first,paragraph_len_first,…,tfid_cnt_2136,tfid_cnt_2137,tfid_cnt_2138,tfid_cnt_2139,tfid_cnt_2140,tfid_cnt_2141,tfid_cnt_2142,tfid_cnt_2143,tfid_cnt_2144,tfid_cnt_2145,tfid_cnt_2146,tfid_cnt_2147,tfid_cnt_2148,tfid_cnt_2149,tfid_cnt_2150,tfid_cnt_2151,tfid_cnt_2152,tfid_cnt_2153,tfid_cnt_2154,tfid_cnt_2155,tfid_cnt_2156,tfid_cnt_2157,tfid_cnt_2158,tfid_cnt_2159,tfid_cnt_2160,tfid_cnt_2161,tfid_cnt_2162,tfid_cnt_2163,tfid_cnt_2164,tfid_cnt_2165,tfid_cnt_2166,tfid_cnt_2167,tfid_cnt_2168,tfid_cnt_2169,tfid_cnt_2170,tfid_cnt_2171,tfid_cnt_2172
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,"""0f2a7d9""",4,4,4,4,4,4,4,3,3,3,2,2,1,0,0,0,0,5,558,4,101,2.5,370.0,3.0,68.25,1,188,2,35,10,1480,12,273,2,405,…,0,0,1,0,5,3,0,0,1,0,2,2,1,0,1,1,1,0,0,0,0,2,2,3,0,3,0,1,1,0,0,0,0,0,0,0,0


In [37]:
len(train_data)

93

In [ ]:
valid_data

In [36]:
len(train_data.columns)

24143

In [34]:
missing_columns = [col for col in ['Unnamed: 0'] if col not in train_data.columns]

In [35]:
missing_columns

['Unnamed: 0']

## persaude2.0

In [8]:
## persaude2.0からtrainデータに含まれていない実績を補填
# persaude2.0
persuade_data = (
    pl.read_csv(path_to.persaude_corpus_2_dir)
        .rename({"holistic_essay_score": config["target"], 
                    "essay_id_comp" : "essay_id"})
        .with_columns(pl.col("full_text").str.split(by="\n\n").alias("paragraph"))
        .select("essay_id", "full_text", config["target"], "paragraph")
)

In [9]:
persuade_data.head()

essay_id,full_text,score,paragraph
str,str,i64,list[str]
"""423A1CA112E2""","""Phones Modern…",3,"[""Phones"", ""Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it."", … ""Phones are fine to use and it's also the best way to come over help. If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving. The news always updated when people do something stupid around that involves their phones. The safest way is the best way to stay safe. ""]"
"""BC75783F96E3""","""This essay wil…",4,"[""This essay will explain if drivers should or should not be able to use electronic devices while operating a vehicle. Using a phone while driving can cause very bad wrecks, because they are putting themselves and other people at risk. People should not be able to use cell phones while operating a vehicle is because bad wrecks, putting others' lives in danger, and may cause death."", ""First of all, wrecks can be caused by looking down at your phone while driving. Most importantly, you should always keep your eyes directly on the road when behind the wheel of a car. On that note, you will have an outrageously car payment just because you were looking down at your phone. Moreover, if you the reason the accident happened the person that was operating the other car can sue you for lots of money, and you will have to pay it. Therefore, if you don't pay whatever the person charges you consequences will be served and that's behind bars."", … ""Must be remembered, always stay focus on the road so you can get to your destination safely. So, you can't cause accidents, put other people lives in danger, and possibly death. Important to realize, do not use your phone while operating a vehicle. At least, wait till you make a safe stop or until you arrive at your destination. ""]"
"""74C8BC7417DE""","""Driving while …",2,"[""Driving while the use of cellular devices"", ""Today, most of the society is thoughtless. Especially new drivers, all driver for that matter do not understand the dangers of looking at a cell phone while behind the wheel."", ""The automobile crashes due to distracted driving are increasing, teens have been the largest age group that was reported to being distracted by cellular usage. I do not agree with people using their cell phones and driving, humans should be smarter to know that if something is going on while they are driving that is so important then pulling off to the side of the road. The society doesn't understand that teens are the main cause for accidents I mean, in two thousand and five there was three hundred and one thousand injuries that were caused in distracted driving related accidents, that doesn't necessarily mean that the driver was texting or using a cellular device. New drivers do not understand that deaths are usually results of an accident. one out of every four car accidents is caused by texting and driving. ""]"
"""A8445CABFECE""","""Phones & Drivi…",3,"[""Phones & Driving"", ""Drivers should not be able to use phones while operating a vehicle. Drivers who used their phone while operating a vehicle are most likely to get into an accident that could be fatal."", … ""In conclusion, drivers should not able to work a vehicle while using their cell phone. Drivers who uses their phones while operating a vehicle and are likely to have an accident then those who don't. ""]"
"""6B4F7A0165B9""","""Cell Phone Ope…",4,"[""Cell Phone Operation While Driving"", ""The abilit

In [16]:
persuade_data.filter(pl.col("full_text")=="")

essay_id,full_text,score,paragraph
str,str,i64,list[str]


In [5]:
create_dataset = CreateDataset(s3_dir, config)

In [6]:
create_dataset.load_dataset()

In [7]:
tmp = create_dataset.Paragraph_Preprocess(create_dataset.train_data)

In [8]:
train_feats = create_dataset.Paragraph_Eng(tmp)

In [10]:
len(create_dataset.train_data)

30430

In [11]:
len(train_feats)

30429

In [12]:
train_feats.head()

,essay_id,paragraph_>0_cnt,paragraph_>50_cnt,paragraph_>75_cnt,paragraph_>100_cnt,paragraph_>125_cnt,paragraph_>150_cnt,paragraph_>175_cnt,paragraph_>200_cnt,paragraph_>250_cnt,...,paragraph_sentence_cnt_kurtosis,paragraph_word_cnt_kurtosis,paragraph_error_num_q1,paragraph_len_q1,paragraph_sentence_cnt_q1,paragraph_word_cnt_q1,paragraph_error_num_q3,paragraph_len_q3,paragraph_sentence_cnt_q3,paragraph_word_cnt_q3
0,000A58BC095E,6,3,3,3,3,3,3,3,3,...,-1.629761,-1.878328,0.0,23.0,1.0,5.0,0.0,416.0,5.0,82.0
1,000BAD50D026,3,3,3,2,2,2,2,2,2,...,-1.500000,-1.500000,1.0,944.0,8.0,178.0,2.0,1141.0,9.0,195.0
2,000E6DE9E817,6,3,3,3,3,3,3,3,3,...,-1.318024,-1.023474,0.0,10.0,1.0,1.0,0.0,464.0,5.0,83.0
3,000d118,1,1,1,1,1,1,1,1,1,...,NaN,NaN,27.0,2640.0,14.0,491.0,27.0,2640.0,14.0,491.0
4,000fe60,5,5,5,5,5,5,5,4,3,...,-1.044379,-1.388460,1.0,235.0,4.0,46.0,1.0,398.0,5.0,77.0


In [13]:
len(train_feats["essay_id"].unique())

30429

In [14]:
len(create_dataset.train_data["essay_id"].unique())

30429

In [16]:
df = create_dataset.train_data

In [17]:
counts = df.groupby("essay_id").count()
duplicates = counts.filter(pl.col("count") > 1)

/tmp/ipykernel_41938/4048290298.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  counts = df.groupby("essay_id").count()
/tmp/ipykernel_41938/4048290298.py:1: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  counts = df.groupby("essay_id").count()


In [19]:
duplicates

essay_id,count
str,u32
"""3.25E+11""",2


In [20]:
df.filter(pl.col("essay_id")=="3.25E+11")

essay_id,full_text,score,paragraph,suppliment_flg
str,str,i64,list[str],str
"""3.25E+11""","""I disagree wit…",4,"[""I disagree with the principal in saying every student should do a extracurricular activity for three reasons."", ""One reason I disagree is because not everyone has the time to do these activities. Like some students might have younger siblings to take care of or afterschool jobs they have to do, such as walking dogs, babysitting, tutoring, and other things. Also some students have responsibilities to do when they get home, maybe they have an elderly person at home, and its their responsibility to take care of that person as soon as school ends."", … ""This is why i disagree with the principal, next time I hope they keep the students in mind before they make a new school rule or if they had talked to the students before making a decision.""]","""1"""
"""3.25E+11""","""Ever since the…",6,"[""Ever since the invention of the telephone, our society has been marked with a series of innovations in the communications field. From texting to social media, technology has made it easier to communicate with each other with little to no latency from anywhere on the planet. One of the most recent applications of communications technology is through the education sector. Some schools offer online video conferences such that students can attend class from home. While many argue that they limit teaching ability and promote longer screen time, online classes provide enough logistical benefits such as cost-effectiveness and safety that they outweigh the criticism."", ""Running a school can be expensive. The school must be able to pay for general utilities, transportation, and safety for each and every student. Taking these factors into account, the school will likely be paying tens of thousands of dollars each day in order to function. This price contrasts heavily with that of online classes as the cost of an online class is typically only dependent on the technology needed to run it. The technology usually needed to run such classes can be broken down to a computer and access to the Internet. Computers have been rapidly becoming cheaper due to innovations in transistor design and other fundamental computational components. Furthermore, they are generally one-time payments as compared to utility costs that must be paid every day. The Internet has experienced vast growth in the last two decades as cell towers and broadband companies have become more abundant. As of right now, we have many developing countries that have gained access to Internet as reductions in costs have made broadband more accessible. If we consider that salaries for teachers remain constant for both in-person and online classes, the price for running online classes seem miniscule when juxtaposed with the logistical costs for running a school."", … ""Criticism against online schools often stem from the belief that communication with a teacher in-person is the best way to learn; however, that statement is becoming more false over time. As schools increasingly gain access to technology, they utilize certain software to increase their learning productivity such as Google Classroom and Microsoft Word. Both of these softwares often require students to have computers. Likewise, online classes share the same demand and produce the same educational result. Hence, the notion that in-person classes has better learning potential than online classes is false. Equipping this information along with the logistical benefits associated with online classes, it is clear to see that online classes provide a benefit for students.""]","""1"""
